# Create a search index in Azure AI Search using the Azure SDK for Python

This notebook steps through creating, loading, and querying an index in Azure AI Search index by calling the azure-search-documents library in the Azure SDK for Python. 

## Install packages and set variables

In [ ]:
! pip install azure-search-documents==11.6.0b1 --quiet
! pip install azure-identity --quiet
! pip install python-dotenv --quiet

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# Provide variables
search_endpoint: str = os.getenv("AZURE_SEARCH_ENDPOINT")
search_api_key: str = os.getenv("AZURE_SEARCH_API_KEY")
index_name: str = "company-quickstart"

## Create an index

In [ ]:
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(search_api_key)

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex
)

# Create a search schema
index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=credential)
fields = [
    SimpleField(name="CompanyId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="StockTicker", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
    SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
    SearchableField(name="Industry", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
    SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),
    SimpleField(name="PubliclyTraded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
    SimpleField(name="FoundedDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
    SimpleField(name="MarketCap", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),
    ComplexField(name="Headquarters", fields=[
        SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
        SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
    ])
]


scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['Tags', 'Headquarters/City', 'Headquarters/Country']}]

# Create the search index=
index = SearchIndex(name=index_name, fields=fields, suggesters=suggester, scoring_profiles=scoring_profiles)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 company-quickstart created


## Create a documents payload

In [ ]:
# Create a documents payload
documents =[
    {
        "@search.action": "upload",
        "CompanyId": "1",
        "": "Contoso Technologies Inc.",
        "StockTicker": "CTSO",
        "Description": "Contoso Technologies is a leading provider of cloud-based solutions, specializing in AI and data analytics for enterprises worldwide.",
        "Description_fr": "Contoso Technologies est un fournisseur leader de solutions cloud, spécialisé dans l'IA et l'analyse de données pour les entreprises du monde entier.",
        "Industry": "Technology",
        "Tags": ["cloud", "AI", "analytics", "enterprise"],
        "PubliclyTraded": True,
        "FoundedDate": "2001-05-15T00:00:00Z",
        "MarketCap": 125.7,
        "Headquarters": {
            "StreetAddress": "123 Innovation Way",
            "City": "Redmond",
            "StateProvince": "WA",
            "PostalCode": "98052",
            "Country": "USA"
        }
    },
    {
        "@search.action": "upload",
        "CompanyId": "2",
        "": "Northwind Foods Corp.",
        "StockTicker": "NWF",
        "Description": "Northwind Foods is a global leader in food processing and distribution, offering a wide range of organic and conventional products.",
        "Description_fr": "Northwind Foods est un leader mondial dans la transformation et la distribution alimentaire, offrant une large gamme de produits biologiques et conventionnels.",
        "Industry": "Food & Beverage",
        "Tags": ["organic", "distribution", "processing", "retail"],
        "PubliclyTraded": True,
        "FoundedDate": "1985-03-10T00:00:00Z",
        "MarketCap": 18.2,
        "Headquarters": {
            "StreetAddress": "456 Market St",
            "City": "Chicago",
            "StateProvince": "IL",
            "PostalCode": "60606",
            "Country": "USA"
        }
    },
    {
        "@search.action": "upload",
        "CompanyId": "3",
        "": "Fabrikam Financial Group",
        "StockTicker": "FBKM",
        "Description": "Fabrikam Financial Group provides innovative banking and investment solutions for individuals and businesses.",
        "Description_fr": "Fabrikam Financial Group propose des solutions bancaires et d'investissement innovantes pour les particuliers et les entreprises.",
        "Industry": "Finance",
        "Tags": ["banking", "investment", "loans", "wealth management"],
        "PubliclyTraded": True,
        "FoundedDate": "1992-07-01T00:00:00Z",
        "MarketCap": 54.3,
        "Headquarters": {
            "StreetAddress": "789 Wall St",
            "City": "New York",
            "StateProvince": "NY",
            "PostalCode": "10005",
            "Country": "USA"
        }
    },
    {
        "@search.action": "upload",
        "CompanyId": "4",
        "": "AdventureWorks Manufacturing",
        "StockTicker": "AWMF",
        "Description": "AdventureWorks specializes in the design and production of high-quality outdoor equipment and sporting goods.",
        "Description_fr": "AdventureWorks est spécialisé dans la conception et la production d'équipements de plein air et d'articles de sport de haute qualité.",
        "Industry": "Manufacturing",
        "Tags": ["outdoor", "equipment", "sports", "design"],
        "PubliclyTraded": False,
        "FoundedDate": "2005-09-15T00:00:00Z",
        "MarketCap": 3.9,
        "Headquarters": {
            "StreetAddress": "321 Industrial Blvd",
            "City": "Denver",
            "StateProvince": "CO",
            "PostalCode": "80216",
            "Country": "USA"
        }
    },
    {
        "@search.action": "upload",
        "CompanyId": "5",
        "": "Woodgrove Healthcare",
        "StockTicker": "WDGV",
        "Description": "Woodgrove Healthcare operates a network of hospitals and clinics, delivering patient-centered care and advanced medical services.",
        "Description_fr": "Woodgrove Healthcare exploite un réseau d'hôpitaux et de cliniques, offrant des soins centrés sur le patient et des services médicaux avancés.",
        "Industry": "Healthcare",
        "Tags": ["hospital", "clinic", "patient care", "medical"],
        "PubliclyTraded": False,
        "FoundedDate": "2010-01-20T00:00:00Z",
        "MarketCap": 7.5,
        "Headquarters": {
            "StreetAddress": "654 Wellness Ave",
            "City": "Houston",
            "StateProvince": "TX",
            "PostalCode": "77030",
            "Country": "USA"
        }
    },
    {
        "@search.action": "upload",
        "CompanyId": "6",
        "": "Litware Energy Solutions",
        "StockTicker": "LTWR",
        "Description": "Litware Energy Solutions develops and manages renewable energy projects, focusing on solar and wind power generation.",
        "Description_fr": "Litware Energy Solutions développe et gère des projets d'énergie renouvelable, axés sur la production d'énergie solaire et éolienne.",
        "Industry": "Energy",
        "Tags": ["renewable", "solar", "wind", "sustainability"],
        "PubliclyTraded": True,
        "FoundedDate": "2012-11-30T00:00:00Z",
        "MarketCap": 22.8,
        "Headquarters": {
            "StreetAddress": "987 Greenway Dr",
            "City": "San Diego",
            "StateProvince": "CA",
            "PostalCode": "92121",
            "Country": "USA"
        }
    }
]

## Upload documents

In [8]:
search_client = SearchClient(endpoint=search_endpoint,
                      index_name=index_name,
                      credential=credential)
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

    index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=credential)

Upload of new document succeeded: True


## Run your first query

In [9]:
# Run an empty query (returns selected fields, all documents)
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='CompanyName,Description',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["CompanyName"])
    print(f"Description: {result['Description']}")


Total Documents Matching Query: 6
1.0
Fabrikam Financial Group
Description: Fabrikam Financial Group provides innovative banking and investment solutions for individuals and businesses.
1.0
Northwind Foods Corp.
Description: Northwind Foods is a global leader in food processing and distribution, offering a wide range of organic and conventional products.
1.0
AdventureWorks Manufacturing
Description: AdventureWorks specializes in the design and production of high-quality outdoor equipment and sporting goods.
1.0
Woodgrove Healthcare
Description: Woodgrove Healthcare operates a network of hospitals and clinics, delivering patient-centered care and advanced medical services.
1.0
Litware Energy Solutions
Description: Litware Energy Solutions develops and manages renewable energy projects, focusing on solar and wind power generation.
1.0
Contoso Technologies Inc.
Description: Contoso Technologies is a leading provider of cloud-based solutions, specializing in AI and data analytics for enter

## Run a term query

In [12]:
# Run a term query
results =  search_client.search(query_type='simple',
    search_text="finance" ,
    select='CompanyName,Description,Tags',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["CompanyName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 1
0.2876821
Fabrikam Financial Group
Description: Fabrikam Financial Group provides innovative banking and investment solutions for individuals and businesses.


## Add a filter

In [32]:
# Add a filter
results = search_client.search(
    search_text="*", 
    select='CompanyId,CompanyName,MarketCap', 
    filter='MarketCap gt 10', 
    order_by='MarketCap desc')

for result in results:
    print("{}: {} - {} MarketCap".format(result["CompanyId"], result["CompanyName"], result["MarketCap"]))

1: Contoso Technologies Inc. - 125.7 MarketCap
3: Fabrikam Financial Group - 54.3 MarketCap
6: Litware Energy Solutions - 22.8 MarketCap
2: Northwind Foods Corp. - 18.2 MarketCap


## Scope a query to specific searchable fields

In [33]:
results = search_client.search(
    search_text="Contoso", 
    search_fields=['CompanyName'], 
    select='CompanyId,CompanyName')

for result in results:
    print("{}: {}".format(result["CompanyId"], result["CompanyName"]))

1: Contoso Technologies Inc.


## Return facets

In [34]:
# Return facets
results = search_client.search(search_text="*", facets=["Industry"])

facets = results.get_facets()

for facet in facets["Industry"]:
    print("    {}".format(facet))

    {'value': 'Energy', 'count': 1}
    {'value': 'Finance', 'count': 1}
    {'value': 'Food & Beverage', 'count': 1}
    {'value': 'Healthcare', 'count': 1}
    {'value': 'Manufacturing', 'count': 1}
    {'value': 'Technology', 'count': 1}


## Look up a document 

In [35]:
# Look up a specific document by ID
result = search_client.get_document(key="3")

print("Name: {}".format(result["CompanyName"]))
print("MarketCap: {}".format(result["MarketCap"]))
print("Industry: {}".format(result["Industry"]))

Name: Fabrikam Financial Group
MarketCap: 54.3
Industry: Finance


## Autocomplete a query

In [37]:
# Autocomplete a query
search_suggestion = 'cl'
results = search_client.autocomplete(
    search_text=search_suggestion, 
    suggester_name="sg",
    mode='twoTerms')

print("Autocomplete for:", search_suggestion)
for result in results:
    print (result['text'])

Autocomplete for: cl
clinic
cloud


## Clean up

If you are finished with this index, you can delete it by running the following lines. Deleting unnecessary indexes frees up space for stepping through more quickstarts and tutorials.

In [ ]:
try:
    result = index_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Confirm the index deletion by running the following script that lists all of the indexes on your search service. If hotels-quickstart is not listed, you've successfully deleted the index and have completed this quickstart.

In [ ]:
try:
    result = index_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)
